In [ ]:
import os
import mediapipe as mp
import cv2
import glob as glob

# files 
data_local = './Data_Local/'
slowSP = glob.glob(data_local + '*oneover60*')
slowSP.sort()
fastSP = glob.glob(data_local + '*oneover250*')
fastSP.sort()
# print
print(slowSP)
print(fastSP)


['./Data_Local\\oneover60_cam1.mp4', './Data_Local\\oneover60_cam2.mp4', './Data_Local\\oneover60_cam3.mp4']
['./Data_Local\\oneover250_cam1.mp4', './Data_Local\\oneover250_cam2.mp4', './Data_Local\\oneover250_cam3.mp4']


In [20]:
import os
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from pathlib import Path

# Initialize MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

class ShutterSpeedDiagnostics:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.slow_files = sorted(glob(os.path.join(data_dir, '*oneover60*')))
        self.fast_files = sorted(glob(os.path.join(data_dir, '*oneover250*')))
        self.results = {}
        
    def calculate_jitter(self, positions, window=5):
        """Calculate jitter using second derivative (acceleration)"""
        if len(positions) < window:
            return np.nan
        
        # Calculate velocity (first derivative)
        velocity = np.diff(positions, axis=0)
        # Calculate acceleration (second derivative)
        acceleration = np.diff(velocity, axis=0)
        # Calculate jitter (magnitude of acceleration)
        jitter = np.linalg.norm(acceleration, axis=1)
        return np.mean(jitter)
    
    def analyze_video(self, video_path, shutter_speed):
        """Analyze a single video file"""
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        camera = Path(video_path).stem.split('_')[-1]
        
        metrics = {
            'shutter_speed': shutter_speed,
            'camera': camera,
            'fps': fps,
            'total_frames': total_frames,
            'hand_visibility': {'left': [], 'right': []},
            'hand_confidence': {'left': [], 'right': []},
            'pose_confidence': [],
            'face_confidence': [],
            'hand_positions': {'left': [], 'right': []},
            'jitter': {'left': [], 'right': []}
        }
        
        with mp_holistic.Holistic(
            static_image_mode=False,
            model_complexity=2,
            enable_segmentation=False,
            refine_face_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        ) as holistic:
            
            frame_idx = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Convert to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process frame
                results = holistic.process(image)
                
                # Analyze hand visibility and confidence
                for hand_side in ['left', 'right']:
                    hand_landmarks = getattr(results, f'{hand_side}_hand_landmarks')
                    
                    if hand_landmarks:
                        metrics['hand_visibility'][hand_side].append(1)
                        
                        # Calculate average confidence for hand landmarks
                        confidences = []
                        for landmark in hand_landmarks.landmark:
                            if hasattr(landmark, 'visibility'):
                                confidences.append(landmark.visibility)
                        
                        if confidences:
                            metrics['hand_confidence'][hand_side].append(np.mean(confidences))
                        else:
                            metrics['hand_confidence'][hand_side].append(1.0)
                        
                        # Store hand position (using wrist as reference)
                        wrist_landmark = hand_landmarks.landmark[0]
                        metrics['hand_positions'][hand_side].append([
                            wrist_landmark.x,
                            wrist_landmark.y,
                            wrist_landmark.z
                        ])
                    else:
                        metrics['hand_visibility'][hand_side].append(0)
                        metrics['hand_confidence'][hand_side].append(0)
                        metrics['hand_positions'][hand_side].append([np.nan, np.nan, np.nan])
                
                # Analyze pose confidence
                if results.pose_landmarks:
                    pose_confidences = []
                    for landmark in results.pose_landmarks.landmark:
                        if hasattr(landmark, 'visibility'):
                            pose_confidences.append(landmark.visibility)
                    
                    if pose_confidences:
                        metrics['pose_confidence'].append(np.mean(pose_confidences))
                    else:
                        metrics['pose_confidence'].append(1.0)
                else:
                    metrics['pose_confidence'].append(0)
                
                # Analyze face confidence
                if results.face_landmarks:
                    # Calculate center face confidence (eyes, nose, mouth area)
                    center_landmarks = [33, 133, 362, 263, 1, 61, 291]
                    face_confidences = []
                    for idx in center_landmarks:
                        if idx < len(results.face_landmarks.landmark):
                            landmark = results.face_landmarks.landmark[idx]
                            if hasattr(landmark, 'visibility'):
                                face_confidences.append(landmark.visibility)
                    
                    if face_confidences:
                        metrics['face_confidence'].append(np.mean(face_confidences))
                    else:
                        metrics['face_confidence'].append(1.0)
                else:
                    metrics['face_confidence'].append(0)
                
                frame_idx += 1
                
                # Print progress
                if frame_idx % 30 == 0:
                    print(f"Processed {frame_idx}/{total_frames} frames for {video_path}")
        
        cap.release()
        
        # Calculate final metrics
        metrics['hand_visibility_percentage'] = {
            'left': (np.mean(metrics['hand_visibility']['left']) * 100),
            'right': (np.mean(metrics['hand_visibility']['right']) * 100)
        }
        
        # FIXED: Handle empty lists and NaN values properly
        metrics['average_hand_confidence'] = {
            'left': np.mean([c for c in metrics['hand_confidence']['left'] if c > 0]) if any(c > 0 for c in metrics['hand_confidence']['left']) else np.nan,
            'right': np.mean([c for c in metrics['hand_confidence']['right'] if c > 0]) if any(c > 0 for c in metrics['hand_confidence']['right']) else np.nan
        }
        
        # Calculate jitter for each hand
        for hand_side in ['left', 'right']:
            positions = np.array(metrics['hand_positions'][hand_side])
            # Remove NaN values
            valid_positions = positions[~np.isnan(positions).any(axis=1)]
            if len(valid_positions) > 10:
                jitter = self.calculate_jitter(valid_positions)
                metrics['jitter'][hand_side] = jitter
            else:
                metrics['jitter'][hand_side] = np.nan
        
        # FIXED: Handle empty lists properly
        metrics['average_pose_confidence'] = np.mean([c for c in metrics['pose_confidence'] if c > 0]) if any(c > 0 for c in metrics['pose_confidence']) else np.nan
        metrics['average_face_confidence'] = np.mean([c for c in metrics['face_confidence'] if c > 0]) if any(c > 0 for c in metrics['face_confidence']) else np.nan
        
        return metrics
    
    def run_analysis(self):
        """Run analysis on all videos"""
        print("Analyzing slow shutter speed videos (1/60)...")
        for video_path in self.slow_files:
            self.results[Path(video_path).stem] = self.analyze_video(video_path, "1/60")
        
        print("\nAnalyzing fast shutter speed videos (1/250)...")
        for video_path in self.fast_files:
            self.results[Path(video_path).stem] = self.analyze_video(video_path, "1/250")
    
    def generate_visualizations(self, output_dir="analysis_output"):
        """Generate the comprehensive visualization and save individual components separately"""
        os.makedirs(output_dir, exist_ok=True)
        
        # Prepare data for visualization
        data = []
        for video_name, metrics in self.results.items():
            for hand_side in ['left', 'right']:
                data.append({
                    'shutter_speed': metrics['shutter_speed'],
                    'camera': metrics['camera'],
                    'hand': hand_side,
                    'visibility_percentage': metrics['hand_visibility_percentage'][hand_side],
                    'average_confidence': metrics['average_hand_confidence'][hand_side],
                    'jitter': metrics['jitter'][hand_side],
                    'video': video_name
                })
        
        df = pd.DataFrame(data)
        
        # 1. Create Comprehensive Visualization (exactly as shown in your image)
        fig = plt.figure(figsize=(20, 16))
        gs = fig.add_gridspec(3, 3, height_ratios=[1, 1, 1], width_ratios=[1, 1, 1],
                             hspace=0.3, wspace=0.3)
        
        fig.suptitle('Shutter Speed Performance Analysis', fontsize=24, fontweight='bold', y=0.98)
        
        # Top row - Hand Visibility Comparison
        ax1 = fig.add_subplot(gs[0, :])
        visibility_pivot = df.pivot_table(index=['camera', 'hand'], 
                                         columns='shutter_speed', 
                                         values='visibility_percentage', 
                                         aggfunc='mean')
        visibility_pivot.plot(kind='bar', ax=ax1, width=0.8, alpha=0.9)
        ax1.set_title('Hand Visibility by Camera and Shutter Speed', fontsize=16, pad=20)
        ax1.set_ylabel('Visibility Percentage (%)', fontsize=12)
        ax1.set_xlabel('')
        ax1.legend(title='Shutter Speed', fontsize=11, title_fontsize=12)
        ax1.grid(axis='y', alpha=0.3, linestyle='--')
        ax1.set_ylim(85, 102)
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
        
        # Middle row visualizations
        ax2 = fig.add_subplot(gs[1, 0])
        df_jitter = df.dropna(subset=['jitter'])
        if not df_jitter.empty:
            camera_jitter = df_jitter.groupby(['camera', 'shutter_speed'])['jitter'].mean().unstack()
            camera_jitter.plot(kind='bar', ax=ax2, width=0.8)
            ax2.set_title('Average Jitter by Camera', fontsize=14)
            ax2.set_ylabel('Jitter Value', fontsize=12)
            ax2.set_xlabel('')
            ax2.legend(title='Shutter Speed', fontsize=10)
            ax2.grid(axis='y', alpha=0.3, linestyle='--')
            ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
        
        ax3 = fig.add_subplot(gs[1, 1])
        if not df_jitter.empty:
            sns.boxplot(data=df_jitter, x='shutter_speed', y='jitter', 
                       hue='hand', ax=ax3, palette=['#3498db', '#e74c3c'])
            ax3.set_title('Jitter Distribution by Hand', fontsize=14)
            ax3.set_ylabel('Jitter Value', fontsize=12)
            ax3.set_xlabel('')
            ax3.legend(title='Hand', fontsize=10)
            ax3.grid(axis='y', alpha=0.3, linestyle='--')
        
        ax4 = fig.add_subplot(gs[1, 2])
        improvements = []
        for camera in df['camera'].unique():
            for hand in ['left', 'right']:
                slow = df[(df['camera'] == camera) & (df['shutter_speed'] == '1/60') & (df['hand'] == hand)]
                fast = df[(df['camera'] == camera) & (df['shutter_speed'] == '1/250') & (df['hand'] == hand)]
                
                if not slow.empty and not fast.empty:
                    vis_improvement = fast['visibility_percentage'].iloc[0] - slow['visibility_percentage'].iloc[0]
                    improvements.append({
                        'camera': camera,
                        'hand': hand,
                        'vis_improvement': vis_improvement
                    })
        
        if improvements:
            imp_df = pd.DataFrame(improvements)
            imp_pivot = imp_df.pivot(index='camera', columns='hand', values='vis_improvement')
            im = ax4.imshow(imp_pivot, cmap='RdYlGn', aspect='auto', vmin=-2, vmax=15)
            ax4.set_title('Visibility Improvement (%)\n(1/250 vs 1/60)', fontsize=14)
            ax4.set_yticks(range(len(imp_pivot.index)))
            ax4.set_yticklabels(imp_pivot.index)
            ax4.set_xticks(range(len(imp_pivot.columns)))
            ax4.set_xticklabels(imp_pivot.columns)
            
            for i in range(len(imp_pivot.index)):
                for j in range(len(imp_pivot.columns)):
                    text = ax4.text(j, i, f'{imp_pivot.iloc[i, j]:.1f}%',
                                   ha='center', va='center', color='black', fontweight='bold')
            
            plt.colorbar(im, ax=ax4, label='Improvement (%)')
        
        # Bottom row - Summary Table
        ax5 = fig.add_subplot(gs[2, :])
        ax5.axis('tight')
        ax5.axis('off')
        
        summary_data = []
        for camera in sorted(df['camera'].unique()):
            for shutter in sorted(df['shutter_speed'].unique()):
                camera_data = df[(df['camera'] == camera) & (df['shutter_speed'] == shutter)]
                
                if not camera_data.empty:
                    left_data = camera_data[camera_data['hand'] == 'left'].iloc[0]
                    right_data = camera_data[camera_data['hand'] == 'right'].iloc[0]
                    
                    summary_data.append([
                        camera,
                        shutter,
                        f"{left_data['visibility_percentage']:.1f}%",
                        f"{right_data['visibility_percentage']:.1f}%",
                        f"{left_data['jitter']:.4f}" if not pd.isna(left_data['jitter']) else "N/A",
                        f"{right_data['jitter']:.4f}" if not pd.isna(right_data['jitter']) else "N/A"
                    ])
        
        table = ax5.table(cellText=summary_data,
                         colLabels=['Camera', 'Shutter Speed', 'Left Visibility', 'Right Visibility', 
                                   'Left Jitter', 'Right Jitter'],
                         cellLoc='center',
                         loc='center')
        
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.auto_set_column_width(col=list(range(len(summary_data[0]))))
        
        for i in range(len(summary_data[0])):
            table[(0, i)].set_facecolor('#4a86e8')
            table[(0, i)].set_text_props(weight='bold', color='white')
            table[(0, i)].set_height(0.06)
        
        for i in range(1, len(summary_data) + 1):
            for j in range(len(summary_data[0])):
                if i % 2 == 0:
                    table[(i, j)].set_facecolor('#f0f0f0')
                table[(i, j)].set_height(0.05)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'comprehensive_performance_analysis.png'), 
                    dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        # 2. Save Individual Components
        
        # 2.1 Hand Visibility by Camera and Shutter Speed
        fig, ax = plt.subplots(figsize=(20, 8))
        visibility_pivot.plot(kind='bar', ax=ax, width=0.8, alpha=0.9)
        ax.set_title('Hand Visibility by Camera and Shutter Speed', fontsize=16, pad=20)
        ax.set_ylabel('Visibility Percentage (%)', fontsize=12)
        ax.set_xlabel('')
        ax.legend(title='Shutter Speed', fontsize=11, title_fontsize=12)
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        ax.set_ylim(85, 102)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'hand_visibility_by_camera.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2.2 Average Jitter by Camera
        fig, ax = plt.subplots(figsize=(8, 8))
        if not df_jitter.empty:
            camera_jitter.plot(kind='bar', ax=ax, width=0.8)
            ax.set_title('Average Jitter by Camera', fontsize=14)
            ax.set_ylabel('Jitter Value', fontsize=12)
            ax.set_xlabel('')
            ax.legend(title='Shutter Speed', fontsize=10)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'jitter_by_camera.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2.3 Jitter Distribution by Hand
        fig, ax = plt.subplots(figsize=(8, 8))
        if not df_jitter.empty:
            sns.boxplot(data=df_jitter, x='shutter_speed', y='jitter', 
                       hue='hand', ax=ax, palette=['#3498db', '#e74c3c'])
            ax.set_title('Jitter Distribution by Hand', fontsize=14)
            ax.set_ylabel('Jitter Value', fontsize=12)
            ax.set_xlabel('')
            ax.legend(title='Hand', fontsize=10)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'jitter_distribution.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2.4 Visibility Improvement
        fig, ax = plt.subplots(figsize=(8, 8))
        if improvements:
            im = ax.imshow(imp_pivot, cmap='RdYlGn', aspect='auto', vmin=-2, vmax=15)
            ax.set_title('Visibility Improvement (%)\n(1/250 vs 1/60)', fontsize=14)
            ax.set_yticks(range(len(imp_pivot.index)))
            ax.set_yticklabels(imp_pivot.index)
            ax.set_xticks(range(len(imp_pivot.columns)))
            ax.set_xticklabels(imp_pivot.columns)
            
            for i in range(len(imp_pivot.index)):
                for j in range(len(imp_pivot.columns)):
                    text = ax.text(j, i, f'{imp_pivot.iloc[i, j]:.1f}%',
                                   ha='center', va='center', color='black', fontweight='bold')
            
            plt.colorbar(im, ax=ax, label='Improvement (%)')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'visibility_improvement.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2.5 Summary Table
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.axis('tight')
        ax.axis('off')
        
        table = ax.table(cellText=summary_data,
                         colLabels=['Camera', 'Shutter Speed', 'Left Visibility', 'Right Visibility', 
                                   'Left Jitter', 'Right Jitter'],
                         cellLoc='center',
                         loc='center')
        
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.auto_set_column_width(col=list(range(len(summary_data[0]))))
        
        for i in range(len(summary_data[0])):
            table[(0, i)].set_facecolor('#4a86e8')
            table[(0, i)].set_text_props(weight='bold', color='white')
            table[(0, i)].set_height(0.06)
        
        for i in range(1, len(summary_data) + 1):
            for j in range(len(summary_data[0])):
                if i % 2 == 0:
                    table[(i, j)].set_facecolor('#f0f0f0')
                table[(i, j)].set_height(0.05)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'summary_table.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"\nVisualizations saved to {output_dir}:")
        print("\nComprehensive:")
        print("1. comprehensive_performance_analysis.png - All metrics in one figure")
        print("\nIndividual Components:")
        print("2. hand_visibility_by_camera.png")
        print("3. jitter_by_camera.png")
        print("4. jitter_distribution.png")
        print("5. visibility_improvement.png")
        print("6. summary_table.png")
    
    def save_detailed_report(self, output_path="./processed/detailed_report.csv"):
        """Save detailed metrics to CSV"""
        data_rows = []
        
        for video_name, metrics in self.results.items():
            # Summarize frame-by-frame data
            frames_with_both_hands = sum(1 for l, r in zip(metrics['hand_visibility']['left'], 
                                                          metrics['hand_visibility']['right']) 
                                        if l == 1 and r == 1)
            
            total_frames = metrics['total_frames']
            both_hands_percentage = (frames_with_both_hands / total_frames) * 100 if total_frames > 0 else 0
            
            row = {
                'video': video_name,
                'shutter_speed': metrics['shutter_speed'],
                'camera': metrics['camera'],
                'fps': metrics['fps'],
                'total_frames': total_frames,
                'left_hand_visibility': metrics['hand_visibility_percentage']['left'],
                'right_hand_visibility': metrics['hand_visibility_percentage']['right'],
                'both_hands_visible': both_hands_percentage,
                'left_hand_confidence': metrics['average_hand_confidence']['left'],
                'right_hand_confidence': metrics['average_hand_confidence']['right'],
                'left_hand_jitter': metrics['jitter']['left'],
                'right_hand_jitter': metrics['jitter']['right'],
                'average_pose_confidence': metrics['average_pose_confidence'],
                #'average_face_confidence': metrics['average_face_confidence']
            }
            data_rows.append(row)
        
        df = pd.DataFrame(data_rows)
        df.to_csv(output_path, index=False)
        
        # Create summary statistics
        summary = df.groupby(['shutter_speed', 'camera']).agg({
            'left_hand_visibility': 'mean',
            'right_hand_visibility': 'mean',
            'left_hand_confidence': 'mean',
            'right_hand_confidence': 'mean',
            'left_hand_jitter': 'mean',
            'right_hand_jitter': 'mean'
        }).round(2)
        
        summary.to_csv(output_path.replace('.csv', '_summary.csv'))
        
        print(f"\nDetailed report saved to: {output_path}")
        print(f"Summary report saved to: {output_path.replace('.csv', '_summary.csv')}")
        
        return df, summary

In [ ]:
# Initialize diagnostics
diagnostics = ShutterSpeedDiagnostics('./Data_Local/')

# Run analysis
diagnostics.run_analysis()

# Generate visualizations
diagnostics.generate_visualizations()

# Save detailed report
df, summary = diagnostics.save_detailed_report()
# save
df.to_csv('./Processed/diagnostics_report.csv', index=False)

# Print summary
print("\nSUMMARY STATISTICS:")
print(summary)

# Print comparative analysis
print("\nSHUTTER SPEED COMPARISON:")
for metric in ['left_hand_visibility', 'right_hand_visibility', 'left_hand_jitter', 'right_hand_jitter']:
    print(f"\n{metric}:")
    comparison = df.pivot_table(values=metric, index='camera', columns='shutter_speed', aggfunc='mean')
    comparison['difference'] = comparison['1/250'] - comparison['1/60']
    comparison['percentage_improvement'] = (comparison['difference'] / comparison['1/60']) * 100
    print(comparison.round(2))

Analyzing slow shutter speed videos (1/60)...
Processed 30/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 60/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 90/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 120/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 150/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 180/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 210/1279 frames for ./Data_Local\oneover60_cam1.mp4
Processed 240/1279 frames for ./Data_Local\oneover60_cam1.mp4
